In [7]:
# project imports
import sys
sys.path.append('/home/mark/Documents/code/drone/sumo/utils')
from runSumo import runSumo
from sumo_loop import sumo_loop

# general imports
import os
import subprocess
import numpy as np
import scipy.linalg
from mpl_toolkits.mplot3d import Axes3D
import pickle
import pandas as pd

# plotting params
from IPython.display import Math
# MATPLOTLIB
import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib as mpl
from matplotlib.gridspec import GridSpec

# colors used
ORANGE_SCALE = ['#FFC999','#FFA04D','#FF7803']
NEW_SCALE = ['#FF9132','#008F00','#8A2BE2']
ORANGE = '#FF9132'
TEAL = '#0598B0'
GREEN = '#008F00'
PURPLE = '#8A2BE2'
GRAY = '#969696'
FIG_WIDTH = 6
FIG_HEIGHT = 8

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
    "font.size": 10,
    "axes.titlesize": 12,
    "axes.spines.right": False,
    "axes.spines.top": False,
    "lines.linewidth": 2
})

In [8]:
# Lets load in some of the simulations from the two lane road
sim_dir = {'2':'/media/mark/Ubuntu/results/new_drone/2_lane/',
           '3':'/media/mark/Ubuntu/results/new_drone/3_lane/',
           '4':'/media/mark/Ubuntu/results/new_drone/4_lane/'}
SAVE_PATHS = {'2':'/home/mark/Documents/code/drone/analysis/2_lane/',
              '3':'/home/mark/Documents/code/drone/analysis/3_lane/',
              '4':'/home/mark/Documents/code/drone/analysis/4_lane/'}
keys = ['2','3','4']

flows = {'2':np.arange(10,3000,10),
         '3':np.arange(10,4500,10),
         '4':np.arange(10,6000,10)}
         
ratios = np.logspace(np.log10(0.001),np.log10(0.1),100)

x_shifts = {'2':50,'3':200,'4':250}

In [9]:
N_smooth = 1800 # 30 minute window
from tqdm import trange

for key in keys:
    data = {}
    data['ratios'] = ratios
    data['flows'] = flows[key]
    all_capacity = []
    for idx_r in trange(100):
        capacity = []
        for f in flows[key]:
            outfile = sim_dir[key] + str(idx_r) + '/' + str(f) + '/outfile.csv'
            df = pd.read_csv(outfile, delimiter=';')
            # get arrival times
            arrival = df.tripinfo_arrival
            time = np.arange(start=0,stop=max(arrival)+1)
            flow_out = np.zeros(int(max(arrival))+1)
            # measure and smooth the flow out
            for t in arrival:
                flow_out[int(t)] += 1
            smooth_flow = np.convolve(flow_out,np.ones(N_smooth)/N_smooth,mode='same')[x_shifts[key]:-1]
            # time = time[0:len(time)-x_shifts[key]-1]
            # compute max capcity
            # print(smooth_flow.shape)
            # break
            capacity.append(smooth_flow[1800]*3600)
        # break
        all_capacity.append(capacity)
    # break
    data['capacity'] = np.asarray(all_capacity)
    pickle.dump(data,open(SAVE_PATHS[key]+'capacity.p','wb'))


 35%|███▌      | 35/100 [03:54<07:15,  6.69s/it]


KeyboardInterrupt: 

In [5]:
# N_smooth = 1800 # 30 minute window
# from tqdm import trange

# for key in keys:
#     data = {}
#     data['ratios'] = ratios
#     data['flows'] = flows[key]
#     all_latency = []
#     for idx_r in trange(100):
#         latency = []
#         for f in flows[key]:
#             outfile = sim_dir[key] + str(idx_r) + '/' + str(f) + '/outfile.csv'
#             df = pd.read_csv(outfile, delimiter=';')
#             # get arrival times
#             vtype = df.tripinfo_vType
#             depart = df.tripinfo_depart
#             arrival = df.tripinfo_arrival
#             trip_time = arrival-depart
#             # compute individual travel times of cars
#             indiv_l = []
#             car_index = 0
#             for t in arrival:
#                 if vtype[car_index] == 'car':
#                     indiv_l.append(trip_time[car_index])
#                 car_index += 1
#             # smooth
#             indiv_l=np.asarray(indiv_l)
#             N_smooth = len(indiv_l)//2
#             smooth_latency = np.convolve(indiv_l,np.ones(N_smooth)/N_smooth,mode='same')
#             # print(smooth_latency.shape)
#             # break
#             # compute latency in minutes
#             latency.append(smooth_latency[N_smooth]/60)
#         # break
#         all_latency.append(latency)
#     # break
#     data['l'] = np.asarray(all_latency)
#     pickle.dump(data,open(SAVE_PATHS[key]+'latency.p','wb'))


  0%|          | 0/100 [00:00<?, ?it/s]

(5,)


In [6]:
N_smooth

2

In [6]:
# CUTOFF = 0.1

# for key in ['2','3','4']:
#     Data = pickle.load(open(SAVE_PATHS[key]+'latency.p','rb'))
#     l = Data['l']
#     ratios = Data['ratios']
#     flows =  Data['flows'] 
#     # pre-process
#     data = []
#     for r_idx in range(len(ratios)):
#         for f_idx in range(l.shape[1]):
#             if l[r_idx,f_idx]<CUTOFF:
#                 pass
#             else:
#                 x = flows[f_idx]*ratios[r_idx]
#                 y = flows[f_idx]
#                 z = l[r_idx,f_idx]
#                 data.append([x,y,z])
#     data = np.asarray(data)
#     # CREATE LINEAR FIT
#     # regular grid covering the domain of the data
#     X,Y = np.meshgrid(np.arange(10,1510,10), flows)
#     XX = X.flatten()
#     YY = Y.flatten()


#     # best-fit linear plane
#     A_lin = np.c_[data[:,0], data[:,1], np.ones(data.shape[0])]
#     C_lin,_,_,_ = scipy.linalg.lstsq(A_lin, data[:,2])    # coefficients

#     # evaluate it on grid
#     print(key, C_lin)


2 [4.12525963e-03 5.35954005e-05 5.98697238e-01]
3 [2.94598421e-03 1.03327147e-04 2.44155661e+00]
4 [1.50397659e-03 8.81281915e-05 3.74193555e+00]
